In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade python-dotenv

In [6]:
from pydantic import BaseModel
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()

# client session
client = OpenAI()

# Structured Outputs with OpenAI APIs

Reference blog: https://openai.com/index/introducing-structured-outputs-in-the-api/

Reference documentation: https://platform.openai.com/docs/guides/structured-outputs/introduction

## Simple Example of Structured Outputs

In [7]:
# Pydantic object
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

# call the model
completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    response_format=CalendarEvent,
)

event = completion.choices[0].message.parsed

In [8]:
event

CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])

## Structuring Chain-of-Thought Responses

In [9]:
class Step(BaseModel):
    explanation: str
    output: str

class MathReasoning(BaseModel):
    steps: list[Step]
    final_answer: str

completion = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format=MathReasoning,
)

math_reasoning = completion.choices[0].message.parsed

In [10]:
math_reasoning

MathReasoning(steps=[Step(explanation='The given equation is 8x + 7 = -23. To solve for x, we need to isolate the variable on one side of the equation. The first step is to remove the constant term on the left side, which is 7. We can do this by subtracting 7 from both sides of the equation.', output='Subtract 7 from both sides: 8x + 7 - 7 = -23 - 7.'), Step(explanation='By subtracting 7 from both sides, we simplify the equation, cancelling out the +7 and obtaining a new equation where the left side only has 8x.', output='8x = -30.'), Step(explanation='Now, the equation is 8x = -30. To solve for x, we need to divide both sides of the equation by the coefficient of x, which is 8.', output='Divide both sides by 8: 8x/8 = -30/8.'), Step(explanation='Dividing both sides by 8 gives us x = -30/8. We can simplify this fraction by finding the greatest common divisor (GCD) of 30 and 8, which is 2.', output='Simplify -30/8 by dividing the numerator and the denominator by 2: x = -15/4.'), Step(ex

## Structured Output with Function Calling

Useful when you are connecting the model to tools, functions, data, etc. in your system.

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_delivery_date",
            "description": "Get the delivery date for a customer's order. Call this whenever you need to know the delivery date, for example when a customer asks 'Where is my package'",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The customer's order ID.",
                    },
                },
                "required": ["order_id"],
                "additionalProperties": False,
            },
        },
        "strict": True, # enables structured outputs
    }
]

messages = []
messages.append({"role": "system", "content": "You are a helpful customer support assistant. Use the supplied tools to assist the user."})
messages.append({"role": "user", "content": "Hi, can you tell me the delivery date for my order?"})

response = client.chat.completions.create(
    model='gpt-4o-2024-08-06',
    messages=messages,
    tools=tools,
)

In [12]:
response

ChatCompletion(id='chatcmpl-9u8xRwOfT2svDitK1gr8mtt34Kme7', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Please provide me with your order ID, and I'll check the delivery date for you.", refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1723167829, model='gpt-4o-2024-08-06', object='chat.completion', service_tier=None, system_fingerprint='fp_2a322c9ffc', usage=CompletionUsage(completion_tokens=18, prompt_tokens=107, total_tokens=125))

## Structured Output with `response_format`

Useful when the model needs to respond to the user in a specified structured way.

In [16]:
response = client.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"}
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "math_response",
            "schema": {
                "type": "object",
                "properties": {
                    "steps": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "prtropeies": {
                                "explanation": {"type": "string"},
                                "output": {"type": "string"}
                            },
                            "required": ["explanation", "output"],
                            "additionalProperties": False
                        }
                    },
                    "final_answer": {"type": "string"}
                },
                "required": ["steps", "final_answer"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)

print(response.choices[0].message.content)

{"steps":[{"explanation":"First, we need to isolate the term with the variable 'x' on one side of the equation. Start by subtracting 7 from both sides of the equation to get rid of the constant term on the left side.","output":"8x + 7 - 7 = -23 - 7"},{"explanation":"Simplify both sides of the equation. The +7 and -7 on the left side cancel each other out, leaving us with just 8x. On the right side, -23 minus 7 equals -30.","output":"8x = -30"},{"explanation":"Now, we need to solve for x by isolating it completely. Since 8x means 8 times x, we can divide both sides of the equation by 8 to solve for x.","output":"(8x)/8 = (-30)/8"},{"explanation":"Simplify the right side of the equation by performing the division. -30 divided by 8 simplifies to -3.75 or can be left as a fraction, -15/4, if preferred.","output":"x = -3.75 or x = -15/4"}],"final_answer":"x = -3.75 or x = -15/4"}
